In [32]:
import cv2 as cvision
import pandas as pd
import numpy as np
import os
import torch
from torch.utils import data
import torchvision
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
class Data:
    def __init__(self,root,csv,batch_size):
        self.root=root
        self.csv=pd.read_csv(csv)
        self.batch_size=batch_size
        self.idx=0
        
        
    def get_next_batch(self):
        x=[]
        y=[]
        tempy=self.csv.iloc[self.idx:self.idx+self.batch_size,:].values
        for row in tempy:
            y.append(row[1:])
            tempx=cvision.imread(os.path.join(self.root,row[0]))
            x.append(tempx)
            
        self.idx=(self.idx+self.batch_size)%len(self.csv)
        return np.array(x),np.array(y)

In [39]:
train_data=Data('train','training.csv',64)
test_data=Data('test','test.csv',64)